In [1]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pandas as pd
import random
import zipfile
import io
import tempfile
import tqdm

In [2]:
#This part is importing the metadata into dfs
df_0 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2016_metadata.csv')
df_1 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2017_metadata.csv')
df_2 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2018_metadata.csv')
df_3 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2019_metadata.csv')
df_4 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2020_metadata.csv')
df_5 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2021_metadata.csv')
df_6 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_2022_metadata.csv')

dfs = pd.concat([df_0,df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True)  # List of DataFrames
print(len(dfs))
dfs.head()

22601041


,BDSPPatientID,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
0,118139267,20160218,Progress Notes,Notes_13246153407_1072726330_20160218.txt
1,120478057,20160829,Progress Notes,Notes_13333565700_1075065164_20160829.txt
2,117769902,20161017,Discharge Summary,Notes_13293992978_1072357331_20161017.txt
3,122117332,20160218,MR AVS Snapshot,Notes_13280349751_1076704882_20160218.txt
4,115708902,20160604,ED Notes,Notes_13255380096_1070297079_20160604.txt


In [3]:
# data filtering of note metadata
#changing date to datetime format
dfs['ContactDate'] = pd.to_datetime(dfs['ContactDate'], format='%Y%m%d')

# post epic only data
start_date = pd.Timestamp('2016-05-01')
end_date = pd.Timestamp('2023-01-01') #We should use these dates too. Only EPIC for MGB. 
complete_df_n = dfs[(dfs['ContactDate'] >= start_date) & (dfs['ContactDate'] < end_date)].reset_index(drop=True)

# filter all notes for discharge notes only
complete_df_discharge = complete_df_n[complete_df_n.InpatientNoteTypeDSC=='Discharge Summary'].reset_index(drop=True)

print(len(complete_df_discharge))
complete_df_discharge.head()
complete_df_discharge.to_csv("complete_df_discharge.csv", index=False)


142161


In [5]:
complete_df_discharge=pd.read_csv("complete_df_discharge.csv")

In [ ]:
# merge note metadata with icd plus group
#change icdplus group to have same date format as note metadata

icdPlus = pd.read_csv('/home/gregory178/Desktop/NAX project/patientIDs_ICD_plus_SDH_MGB.csv')
icdPlus['ShiftedContactDTS'] = pd.to_datetime(icdPlus['ShiftedContactDTS'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
icdPlus['ShiftedContactDTS'] = pd.to_datetime(icdPlus['ShiftedContactDTS'])
icdPlus['BDSPPatientID'] = icdPlus['BDSPPatientID'].astype('int64')

# merge
final_cpt_pos_notes = pd.merge(complete_df_discharge, icdPlus, on=['BDSPPatientID'], how='inner')

# filter for icd within 30d of note creation
# Calculate the absolute difference in days between the contact dates
date_difference = abs((final_cpt_pos_notes['ContactDate'] - final_cpt_pos_notes['ShiftedContactDTS']).dt.days)
# Filter the DataFrame to include only rows where the difference is less than 30 days
ICDPositiveDS = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)

ICDPositiveDS.to_csv("ICD_Postive_DS_before_500.csv", index=False)

In [3]:
ICDPositiveDS = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/ICD_Postive_DS_before_500.csv')

In [ ]:
# create cpt positive list
# # read the thunderpack file
reader = ThunderReader('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_cpt_1m_MGB')

# reminder: ICD+ group is defined as patients who got at least one SDH-related ICD
dfs = []

for i in tqdm(range(1, (len(list(reader.keys()))+1))):    # create a for loop to loop over all keys: ICD_partition_1, ICD_partition_2, ... You can find all keys by `print(list(reader.keys()))`
    df = reader[f'cpt_partition_{i}']
    df = df[df.CPT.astype(str).str.contains('^(?:70450|70460|70470|70551|70552|70553)')]  # for example, you want to filter and keep only ICDs that starts with G20 or 332.  '^(?:G20|332)' is a regular expression (regex).
    dfs.append(df)
df_cpt = pd.concat(dfs, axis=0, ignore_index=True)  # concatenate all filtered dataframes from all partitions into one dataframe
# keep unique patient IDs
# output cpt positive list to csv
df_cpt = df_cpt[['BDSPPatientID','CPT','StartDTS']].drop_duplicates(ignore_index=True)
df_cpt.to_csv('patientIDs_CPT_HeadMRICT_MGB.csv', index=False)    # save as a CSV file


In [5]:
# merge cpt list same way as with icds
# merge note metadata with icd plus group
#change cpt group to have same date format as note metadata
cpt = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/patientIDs_CPT_HeadMRICT_MGB.csv')
cpt['StartDTS'] = pd.to_datetime(cpt['StartDTS'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
cpt['StartDTS'] = pd.to_datetime(cpt['StartDTS'])
cpt['BDSPPatientID'] = cpt['BDSPPatientID'].astype('int64')

# merge
final_cpt_pos_notes = pd.merge(ICDPositiveDS, cpt, on=['BDSPPatientID'], how='inner')
final_cpt_pos_notes['ContactDate'] = pd.to_datetime(final_cpt_pos_notes['ContactDate'])
final_cpt_pos_notes['StartDTS'] = pd.to_datetime(final_cpt_pos_notes['StartDTS'])
# filter for icd within 30d of note creation
# Calculate the absolute difference in days between the contact dates
date_difference = abs((final_cpt_pos_notes['ContactDate'] - final_cpt_pos_notes['StartDTS']).dt.days)
# Filter the DataFrame to include only rows where the difference is less than 30 days
ICDPositiveDS_CPT = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)

ICDPositiveDS_CPT.to_csv("ICD_Postive_DS_CPT_before_500.csv",index=False)

#filter for dates (just like icds)

In [4]:
# run haoqi's code to get text and filter text for these notes
def select_random_note_zip(df, patient_id, base_zip_path):
    # Given patient ID read the notes from the zip file and then select one random note from notes longer than 500 words. 

    # df is the meta file with "BDSPPatientID", "ContactDate", "FileName"
    # patient_notes is the subset within df, for this patient only
    patient_notes = df[df['BDSPPatientID'] == patient_id]#patient notes is the metadata for this patient 
    valid_notes = []
    zip_file = None

    for _, row in patient_notes.iterrows():
        year = row['DeidentifiedName'].split('_')[-1][:4]
        file_path = f'mgb_notes_{year}/{year}/' + row['DeidentifiedName']
        zip_path = f'{base_zip_path}mgb_notes_{year}.zip'
        try:
            if zip_file is None or zip_file.filename != zip_path:
                if zip_file is not None:
                    zip_file.close()
                zip_file = zipfile.ZipFile(zip_path, 'r')
            
            with zip_file.open(file_path) as file_obj:
                try:
                    contents = file_obj.read().decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        contents = file_obj.read().decode('latin1')
                    except UnicodeDecodeError:
                        contents = file_obj.read().decode('iso-8859-1')
                #Here we sort through based on word count. 
                word_count = len(contents.split())
                print(f"Length of contents for {file_path}: {word_count}")
                if word_count >= 500:
                    valid_notes.append((row['BDSPPatientID'], contents, row['ContactDate'], word_count, row['DeidentifiedName'], 'MGB'))
        except (KeyError, FileNotFoundError, zipfile.BadZipFile) as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    if zip_file is not None:
        zip_file.close()
    if valid_notes:
        selected_note = random.choice(valid_notes) #Randomly select one note from notes longer than 500 words.
        return selected_note
    else:
        return None

base_zip_path= '/home/gregory178/Desktop/Zip_Files/'
ICDPL_df=pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/ICD_Postive_DS_CPT_before_500.csv')
FULLICDPL_list = ICDPL_df['BDSPPatientID'].tolist()  # this is the list of all patients with at least one ICD code

# ICDMI_df=pd.read_csv('patientIDs_ICD_minus_SDH_MGB.csv')
# FULLICDMI_list = ICDMI_df['BDSPPatientID'].tolist()

#IS HE MAKING A TABLE HERE?
columns = ['BDSPPatientID', 'note_text', 'CreateDate', 'length', 'deidentifiedname', 'hospital']

#Making ICD + and - groups for MGB
ICDPL_MGB = pd.DataFrame()
ICDMI_MGB = pd.DataFrame()



In [1]:
# now we are doing ICD+ and discharge notes
# output ++ group to a csv
counterPL = 0
added_patient_idsPL = []
print(len(added_patient_idsPL))
print(added_patient_idsPL)

# to ensure the reproducibility
random.seed(2024)

while len(ICDPL_MGB) < 750:
    randpatientID = random.choice(FULLICDPL_list) # this means to select one patient randomly, however, it makes it not reproducible
    UPDATED_LEN_GROUP=len(FULLICDPL_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsPL:
        rnote2 = select_random_note_zip(complete_df_discharge, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDPL_MGB = pd.concat([ICDPL_MGB, pd.DataFrame([rnote2])], ignore_index=True) #, columns=columns
            added_patient_idsPL.append(randpatientID)
            FULLICDPL_list.remove(randpatientID)
            counterPL += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterPL}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDPL_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDPL_list.remove(randpatientID)
    if not FULLICDPL_list:
        print("No more patient IDs left to process")

ICDPL_MGB.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDPL_MGB.to_csv('MGB_sampling_cohort_ICD+_discharge_notes.csv', index=False)

0
[]


NameError: name 'random' is not defined

In [5]:
### create the minus sampling cohort

# the minus cohort can be defiend as the difference between the whole dataset and the plus cohort
# and then, we will limit to patients with notes with >=500 words

# run the upper block to generates complete_df_discharge, if you haven't done it

# we want to further limit the full set to people without ICD (ICD-)
icd_minus_df = pd.read_csv('MGB_ICD_minus.csv')
full_patient_list = sorted( set(complete_df_discharge.BDSPPatientID) & set(icd_minus_df.BDSPPatientID) )

# FULLICDMI_list = full_patient_list - FULLICDPL_list
# method 1:
#FULLICDMI_list = [x for x in full_patient_list if x not in FULLICDPL_list]

# method 2:
FULLICDMI_list = sorted( set(full_patient_list) - set(FULLICDPL_list) )   # <-- Haoqi prefers this way, since the result has unique patientIDs.

In [7]:
### now we are doing ICD- and discharge notes

counterMI = 0
added_patient_idsMI = []
print(len(added_patient_idsMI))
print(added_patient_idsMI)
random.seed(13)

while len(ICDMI_MGB) < 750:
    randpatientID = random.choice(FULLICDMI_list) # this means to select one patient randomly, however, it makes it not reproducible
    UPDATED_LEN_GROUP=len(FULLICDMI_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsMI:
        rnote2 = select_random_note_zip(complete_df_discharge, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDMI_MGB = pd.concat([ICDMI_MGB, pd.DataFrame([rnote2])], ignore_index=True) #, columns=columns
            added_patient_idsMI.append(randpatientID)
            FULLICDMI_list.remove(randpatientID)
            counterMI += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterMI}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDMI_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDMI_list.remove(randpatientID)
    if not FULLICDMI_list:
        print("No more patient IDs left to process")

ICDMI_MGB.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDMI_MGB.to_csv('MGB_sampling_cohort_ICD_minus_discharge_notes.csv', index=False)

0
[]
Updated length list 37264.
Length of contents for mgb_notes_2016/2016/Notes_13277290853_998475736_20160506.txt: 269
Length of contents for mgb_notes_2016/2016/Notes_13279973881_1021790777_20160519.txt: 263
Length of contents for mgb_notes_2017/2017/Notes_13318855192_1545875132_20170613.txt: 4038
Length of contents for mgb_notes_2017/2017/Notes_13318855192_1544542624_20170612.txt: 4022
Length of contents for mgb_notes_2018/2018/Notes_13372450903_2098836154_20180812.txt: 1947
Length of contents for mgb_notes_2021/2021/Notes_13546441773_6114886930_20210814.txt: 256
Added note for patient 116291874. Total notes added: 1
Updated length list 37263.
Length of contents for mgb_notes_2016/2016/Notes_13352426779_1384997463_20160721.txt: 2254
Added note for patient 114884810. Total notes added: 2
Updated length list 37262.
Length of contents for mgb_notes_2016/2016/Notes_13269321066_1273020171_20160626.txt: 666
Added note for patient 114130469. Total notes added: 3
Updated length list 37261.